<a href="https://colab.research.google.com/github/MoronSlayer/Deep-Learning-Projects/blob/learner/BERT_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 111054, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (75/75), done.
^C


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 25.8 MB/s 
     |████████████████████████████████| 163 kB 54.0 MB/s 
     |████████████████████████████████| 6.6 MB 65.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 26.6 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 115 kB 74.3 MB/s 
     |████████████████████████████████| 127 kB 69.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip show transformers

Name: transformers
Version: 4.22.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: tqdm, huggingface-hub, requests, importlib-metadata, packaging, regex, tokenizers, pyyaml, filelock, numpy
Required-by: 


In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext",'wikitext-2-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['text'],
    num_rows: 36718
})

In [ ]:
dataset['train'][36717]

{'text': ''}

In [ ]:
import pandas as pd 
df_train = pd.DataFrame(dataset['train'])


In [ ]:
df_train.to_csv('/content/sample_data/train.txt')

In [ ]:
len(df_train)

36718

In [ ]:
from transformers import BertTokenizer
bert_cased_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import BertConfig, BertForPreTraining
config = BertConfig()
model = BertForPreTraining(config)

In [ ]:
config


BertConfig {
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [ ]:
# -*- coding: utf-8 -*-
"""transformer custom.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1FnIvOB0pZcYxCn_2q5jxNRJ4TmksaM9C
"""

import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
import numpy as np
from torch.utils.data import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging
from sklearn.model_selection import train_test_split


DEPRECATION_WARNING = (
    "This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets "
    "library. You can have a look at this example script for pointers: {0}"
)
logger = logging.get_logger(__name__)
class TextDatasetForNextSentencePrediction3(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        short_seq_probability=0.5,
        nsp_probability=0.5,


    ):
        warnings.warn(
            DEPRECATION_WARNING.format(
                "https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py"
            ),
            FutureWarning,
        )
    
        if not os.path.isfile(file_path):
            raise ValueError(f"Input file path {file_path} not found")

        self.short_seq_probability = short_seq_probability
        self.nsp_probability = nsp_probability

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            f"cached_nsp_{tokenizer.__class__.__name__}_{block_size}_{filename}",
        )
        #print('directory', directory, 'filename=', filename)

        self.tokenizer = tokenizer

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.

        
        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.
        ff=2

        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not overwrite_cache and ff==3:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.documents = [[]]
                with open(file_path, encoding="utf-8") as f:
                    while True:
                        line = f.readline()
                        if not line:
                            break
                        line = line.strip()

                        # Empty lines are used as document delimiters
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([])
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)

                logger.info(f"Creating examples from {len(self.documents)} documents.")
                self.examples = []
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index, block_size)

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    f"Saving features into cached file {cached_features_file} [took {time.time() - start:.3f} s]"
                )


    def create_examples_from_document(self, document: List[List[int]], doc_index: int, block_size: int):
        ln_doc = len(document)
        index = np.arange(ln_doc-1)
        labels = np.zeros(ln_doc)



        correct_index, incorrect_index = train_test_split(index, test_size=self.nsp_probability, random_state=42)
        print(correct_index , 'correct')
        print(incorrect_index, 'incorr')

        for i in correct_index:
            input_ids = self.tokenizer.build_inputs_with_special_tokens(document[i], document[i+1])
            token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(document[i], document[i+1])

            example = {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "next_sentence_label": torch.tensor(0, dtype=torch.long),
                    }
            self.examples.append(example) 

        for i in incorrect_index:
            token_b = random.randint(0, ln_doc-1)
            if token_b == i+1:
                if ln_doc - i > i - 0:
                    token_b = random.randint(i+1, ln_doc-1)
                else:
                    token_b = random.randint(0, i)
            input_ids = self.tokenizer.build_inputs_with_special_tokens(document[i], document[token_b])
            token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(document[i], document[token_b])
            labels[i]=1
                
            example = {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "next_sentence_label": torch.tensor(1, dtype=torch.long),
                    }

            self.examples.append(example)           

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [ ]:
from transformers import TextDatasetForNextSentencePrediction
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=bert_cased_tokenizer,
    file_path= "/content/dataset",
    block_size = 25,
    nsp_probability=0.5
)


Creating features from dataset file at /content
Creating examples from 1 documents.
Saving features into cached file /content/cached_nsp_BertTokenizer_25_dataset [took 0.002 s]


In [ ]:
list(dataset)

[{'input_ids': tensor([  101,  2256,  2783,  4772,  2291,  2323,  2022, 17051,  2000, 25845,
           1996,  3446,  1997,  5082,  1999,  2256,  2492,  1012,   102,  2023,
           2965, 29494,  1996,  3177,  2012,  2029,  2047,  4784,  1998,  3463,
           2024, 10573,  1010,  4487, 11393, 26972,  1010,  1998, 16330,  1012,
            102]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  'next_sentence_label': tensor(0)},
 {'input_ids': tensor([  101,  2023,  2036,  2965,  7163,  4328,  6774,  1996,  3815,  1997,
           2051,  2169,  1997,  2149, 15970, 23208,  2060,  2111,  1005,  1055,
           2147,  2083, 15252,  1998,  9033, 26169,  2083,  1996,  3906,  1012,
            102,  1037,  2350,  3277,  2003,  2008,  2256,  2783,  2291,  1010,
           2007,  2049,  7902,  2006,  3811,  1011, 13228,  9281,  1010,  2003,
           3811, 25352,  2114,  952

In [ ]:
not_nsp=0
nsp=0
for i in range(len(dataset)):
  if dataset[i]['next_sentence_label'].tolist()==1:
    not_nsp+=1
  else:
    nps+=1

In [ ]:
print

0

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

example = "Newton invents calculus."

print({x : tokenizer.encode(x, add_special_tokens=False) for x in example.split()})

{'Newton': [8446], 'invents': [1999, 15338, 2015], 'calculus.': [19276, 1012]}


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
example = "An apple falls under the tree."
print({x : tokenizer.encode(x, add_special_tokens=False) for x in example.split()})

{'An': [2019], 'apple': [6207], 'falls': [4212], 'under': [2104], 'the': [1996], 'tree.': [3392, 1012]}


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
example = "A quick brown fox jumps over the lazy dog."
print({x : tokenizer.encode(x, add_special_tokens=False) for x in example.split()})

{'A': [1037], 'quick': [4248], 'brown': [2829], 'fox': [4419], 'jumps': [14523], 'over': [2058], 'the': [1996], 'lazy': [13971], 'dog.': [3899, 1012]}


In [ ]:
for i in range(8):
  print(len(bert_cased_tokenizer.convert_ids_to_tokens(dataset[i]['input_ids'].tolist())))

49
70
70
70
70
49
46
62


In [ ]:
dataset[7]['next_sentence_label']

tensor(1)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_cased_tokenizer, 
    mlm=True,
    mlm_probability= 0.15,
    
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "/content/drive/MyDrive/Colab Notebooks/BERT/Models/Training",
    # overwrite_output_dir=True,
    num_train_epochs=50,
    per_gpu_train_batch_size= 16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/BERT/Models")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_w

masked_lm_loss tensor(7.4117, device='cuda:0', grad_fn=<NllLossBackward0>) next_sentence_loss tensor(1.3436, device='cuda:0', grad_fn=<NllLossBackward0>) total_loss tensor(8.7553, device='cuda:0', grad_fn=<AddBackward0>)
nsp_labels tensor([0, 0, 1, 1, 0, 1, 1], device='cuda:0')


Step,Training Loss


masked_lm_loss tensor(6.9051, device='cuda:0', grad_fn=<NllLossBackward0>) next_sentence_loss tensor(1.2789, device='cuda:0', grad_fn=<NllLossBackward0>) total_loss tensor(8.1840, device='cuda:0', grad_fn=<AddBackward0>)
nsp_labels tensor([0, 1, 1, 1, 0, 1, 1], device='cuda:0')
masked_lm_loss tensor(6.8750, device='cuda:0', grad_fn=<NllLossBackward0>) next_sentence_loss tensor(1.3438, device='cuda:0', grad_fn=<NllLossBackward0>) total_loss tensor(8.2188, device='cuda:0', grad_fn=<AddBackward0>)
nsp_labels tensor([0, 1, 0, 1, 0, 1, 1], device='cuda:0')
masked_lm_loss tensor(7.2960, device='cuda:0', grad_fn=<NllLossBackward0>) next_sentence_loss tensor(0.7605, device='cuda:0', grad_fn=<NllLossBackward0>) total_loss tensor(8.0566, device='cuda:0', grad_fn=<AddBackward0>)
nsp_labels tensor([0, 1, 1, 0, 0, 1, 1], device='cuda:0')
masked_lm_loss tensor(7.1286, device='cuda:0', grad_fn=<NllLossBackward0>) next_sentence_loss tensor(0.7247, device='cuda:0', grad_fn=<NllLossBackward0>) total_los



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/BERT/Models
Configuration saved in /content/drive/MyDrive/Colab Notebooks/BERT/Models/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/BERT/Models/pytorch_model.bin


In [ ]:
list(dataset)

[{'input_ids': tensor([  101,  4784,  2008,  2735,  2041,  2000,  2022,  3811,  6383,  2024,
           2823,  2218,  2039,  2005,  2706,  1006,  2065,  2025,  2086,  1007,
           1999, 15252, 16405, 28921,  7062,  1010,  3391,  2065,  2027,  5478,
           2195,  2086,  2000,  2272,  2000, 16736,  1006,  2045,  2024,  1037,
           2261,  3297,  4973,  1010,  3855,  1007,  1012,   102,  1996, 15012,
           1999,  2256,  4772,  2291,  2003, 18068,  1996,  5082,  1997,  2256,
           2492,  1012,  2009,  3084,  5082,  4297, 28578, 21050,  1012,  1998,
           2009,  3084,  2256,  9281,  5399, 11771,  1012,   102]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]),
  'next_sentence_label': tensor(0)},
 {'input_ids': tensor([  101

In [ ]:
import torch
x = torch.randn( 8, 70)

x.shape


torch.Size([8, 70])

In [ ]:
y = torch.ones([8])

y[2]=0
y[3]=0
y.shape

torch.Size([8])

In [ ]:
y

tensor([[1.],
        [1.],
        [0.],
        [0.]])

In [ ]:

z= torch.mul(x, y)

In [ ]:
z.shape

torch.Size([4, 2])

In [ ]:
for i in range(len(y)):
  if y[i] == 0:
    x[i]= x[i]*0 - 100
    
    

In [ ]:
x[3]

tensor([-100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100., -100.])

In [ ]:
x.view(-1, 2).shape

torch.Size([8, 2])

In [ ]:
CrossEntropyLoss()